In [8]:
%load_ext autoreload
%autoreload 1
import numpy as np
import pandas as pd
from d3IpyPlus.d3IpyPlus import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### The python kernel used is py2.7

All visualizations are based on [D3IPyPlus](https://github.com/maclandrol/d3IpyPlus), a small module that ports D3plus to the Notebook.

## Déficit commercial

In [9]:
cols = [u'Year', u'Trade Flow Code', u'Trade Flow',
       u'Reporter', u'Netweight (kg)', u'Trade Value (US$)']

df = from_csv('raw_csv_data/comtrade_benin.csv')
df = df[cols]
df = df.loc[df['Trade Flow Code'] != 4]

lgd =  {"font": {
        "size": 12
      },"data":False, "size": [20, 50]}
lplot = LinePlot(id="Trade Flow", x={'value':'Year', 'grid':False,  'ticks':{'labels':range(1998, 2018, 2)}}, y={'value':'Trade Value (US$)', 'grid':False}, width=600, legend=lgd)

lplot.draw(df)

<IPython.core.display.Javascript object>

In [10]:
### Exportation du Bénin

In [11]:
cols = [u'Year', u'Trade Flow Code', u'Trade Flow',
       u'Reporter', u'Commodity Code',
       u'Commodity', u'Netweight (kg)', u'Trade Value (US$)']

df = from_csv('raw_csv_data/comtrade_all_2016.csv')
df = df[cols]
# remove total
def get_commodity_map(df):
    cc = df['Commodity Code']
    cc2 = df['Commodity']
    new_code = []
    ind = []
    code_map = {}
    for i,c in enumerate(cc):
        code_map[str(c)] = str(cc2[i])
        
    for i,c in enumerate(cc):
        if len(c) == 6:
            ind.append(i)
        c = c.ljust(6)
        c1, c2, c3 = [c[i:i+2].strip() for i in range(0, len(c), 2)]
        new_code.append((code_map.get(c1, ""), code_map.get(c1+c2, ""), code_map.get(c1+c2+c3, "")))
    new_code = pd.DataFrame(new_code, columns=['L1', 'L2', 'L3'])    
    df = pd.concat([df, new_code], axis=1)
    df = df.iloc[ind]
    df = df.loc[df['Commodity Code'] != 'TOTAL']
    #print code_map
    return df

df = get_commodity_map(df)
df_export = df.loc[df['Trade Flow Code']==2]

In [4]:
tlp =  {"children":False}
tmap = TreeMap(id=["L1", "L2"], value="Trade Value (US$)", color="L1", width=600, tooltip=tlp)
#print tmap.dump_html(df)
tmap.draw(df_export)

<IPython.core.display.Javascript object>

### Importations du Bénin

In [14]:
df_import = df.loc[df['Trade Flow Code'].isin([1,4])]
#print len(df_import["L1"].unique())
tmap = TreeMap(id=["L1", "L2"], value="Trade Value (US$)", color="L1", width=600, tooltip=tlp)
tmap.draw(df_import)


<IPython.core.display.Javascript object>

### Importation de riz dans quelques pays

In [15]:
import glob
rice_per_country = glob.glob('raw_csv_data/[0-9]_rice.csv')

In [16]:
dflist = [pd.read_csv(x)  for x in rice_per_country]

In [26]:
adf = dflist[0]
adf = adf.append(dflist[1:], ignore_index=True)
#adf = pd.concat(dflist)
adf = adf.loc[adf['Trade Flow Code']==1]
cols = [u'Classification',
       u'Reporter', u'Netweight (kg)', u'Trade Value (US$)']

adf = adf[cols]

adf.columns = [u'Classification',
       u'Pays', u'Quantité (kg)', 'Riz importé ($US)']
adf = adf.reset_index(drop=True)


In [18]:
import pycountry
import wbpy
import d3IpyPlus
api = wbpy.IndicatorAPI()
iso_country_codes = []
total_population = "SP.POP.TOTL"

for d in adf['Pays']:
    try:
        pc = pycountry.countries.get(name=d)
    except:
        pc = pycountry.countries.get(alpha_3=d)
    iso_country_codes.append(pc.alpha_2)
dataset = api.get_dataset(total_population, iso_country_codes, date="2016")
pop = dataset.as_dict()
cmap = {'DZA': 'af',
'BEL' :  'eu',
'GHA': 'af',
'MEX': 'na',
'BRA': 'sa',
'CAN' : 'na',
'FRA' : 'eu',
'SEN' : 'af',
'USA' :'na',
'BEN' : 'af',
'NER' : 'af',
'NGA' : 'af',
'TGO' : 'af',
'BFA' : 'af',
'GRC' : 'eu',
'JPN' : 'as',
'RWA' : 'af',
'ZWE' :'af',
'TUN' :'af'}

In [19]:
new_col = []
for d in adf['Pays']:
    try:
        pc = pycountry.countries.get(name=d)
    except:
        pc = pycountry.countries.get(alpha_3=d)
    new_col.append((pc.alpha_3, int(pop[pc.alpha_2]["2016"])))
    

In [20]:
new_col = pd.DataFrame(new_col, columns=['Country', 'Population'])
adfx = pd.concat([adf, new_col], axis=1)
adfx

Classification          Pays  Quantité (kg)  Riz importé ($US) Country  \
0              H0       Algeria   1.105467e+08           60098878     DZA   
1              H0       Belgium   4.303660e+08          263263430     BEL   
2              H0         Ghana   6.985075e+08          287067234     GHA   
3              H0        Mexico   9.294821e+08          322824399     MEX   
4              H0        Brazil   7.645132e+08          288688501     BRA   
5              H0        Canada   4.651583e+08          289749739     CAN   
6              H0        France   5.720344e+08          447111575     FRA   
7              H0       Senegal   9.737593e+08          325604432     SEN   
8              H0           USA   8.485936e+08          714492803     USA   
9              H0         Benin   1.509421e+09          773532333     BEN   
10             H0         Niger   4.114880e+08          152315821     NER   
11             H0       Nigeria   8.731736e+07           71515876     NGA   
12             H0          Togo   1.400344e+08           18388420     TGO   
13             H0  Burkina Faso   4.838273e+08          107114485     BFA   
14             H0        Greece   1.952214e+07           14432158     GRC   
15             H0         Japan   6.859430e+08          439634212     JPN   
16             H0        Rwanda   5.309418e+07           27112854     RWA   
17             H0      Zimbabwe   1.836599e+08          109862170     ZWE   
18             H0       Tunisia   2.218981e+07            8307659     TUN   

    Population  
0     40606052  
1     11338476  
2     28206728  
3    127540423  
4    207652865  
5     36264604  
6     66892205  
7     15411614  
8    323127513  
9     10872298  
10    20672987  
11   185989640  
12     7606374  
13    18646433  
14    10770521  
15   126994511  
16    11917508  
17    16150362  
18    11403248

In [22]:
geo = d3IpyPlus.d3IpyPlus._GeoMap(id="Country", value="Imported Rice ($US)", text="Pays", tooltip=['Pays', "Population", "Imported Rice ($US)", "Quantity (kg)"])
# Dumpt the html code for the geo map
print geo.dump_html(adfx)


        <script src='//d3plus.org/js/d3.js' type='text/javascript'></script>
	<script src='//d3plus.org/js/topojson.js' type='text/javascript'></script>
	<script src='//d3plus.org/js/d3plus.js' type='text/javascript'></script>
        <div id='d3viz_9'></div>
        <style>
        div#d3viz_9{
             width: 300px;
             height: 300px;
        }
        
        </style>
        <script>
            
        (function (){

            

            var viz_data = [{"Classification":"H0","Pays":"Algeria","Quantit\u00e9 (kg)":110546678.0,"Riz import\u00e9 ($US)":60098878,"Country":"DZA","Population":40606052},{"Classification":"H0","Pays":"Belgium","Quantit\u00e9 (kg)":430366016.0,"Riz import\u00e9 ($US)":263263430,"Country":"BEL","Population":11338476},{"Classification":"H0","Pays":"Ghana","Quantit\u00e9 (kg)":698507472.0,"Riz import\u00e9 ($US)":287067234,"Country":"GHA","Population":28206728},{"Classification":"H0","Pays":"Mexico","Quantit\u00e9 (kg)":929482097.0,"Riz i

### Comparaison des importations entre le Bénin, le Togo et le Nigéria

In [27]:
df = pd.read_csv('raw_csv_data/benin_nigeria_rice.csv')
cols = [u'Year', u'Trade Flow Code', u'Trade Flow',
       u'Reporter',  u'Commodity', u'Netweight (kg)', u'Trade Value (US$)']
df = df[cols]
df = df.loc[(df['Trade Flow Code']==1)  & (df['Year']>=2007)]


In [28]:
dfexport = pd.read_csv('raw_csv_data/benin_export_nigeria_rice.csv')
dfexport = dfexport[cols]
dfexport = dfexport.loc[(dfexport['Year']>=2007) & (dfexport['Trade Flow']=='Export')]
dfexport = dfexport[['Year', 'Trade Flow Code', "Reporter", 'Netweight (kg)', 'Trade Value (US$)']]
dfexport['Trade Flow Code'].loc[dfexport['Trade Flow Code']==2]= 5
dfexport

Year  Trade Flow Code Reporter  Netweight (kg)  Trade Value (US$)
0   2013                5    Benin        78266628            9494202
2   2014                5    Benin        18291000            3983133
5   2007                5    Benin         1000000             304073
6   2008                5    Benin        27253920           10739020
8   2009                5    Benin       140402321           37949290
10  2010                5    Benin       552377525           91040270
12  2011                5    Benin        60110110            9118621
13  2012                5    Benin        23223413            2271430

In [29]:
lgd =  {"font": {
        "size": 12
      },"data":False, "size": [20, 40]}
lp = BarPlot(id="Reporter", x="Year", y="Trade Value (US$)", tooltip=['Trade Value (US$)', 'Netweight (kg)'], legend=lgd, container_id="rice_import")
lp.draw(df)

<IPython.core.display.Javascript object>

### Importation, exportation, production et Besoin en riz du Bénin 

In [150]:
cols = ["Commodity_Code","Commodity_Description","Country_Code","Country_Name","Année","Calendar_Year","Month","Attribute_ID","Attribute_Description","Unit_ID","Unit_Description","Total en tonne"]
col_ind = [3,4,8,11]
rice_bj = pd.read_csv('raw_csv_data/rice_usda_benin.csv', header=None, usecols=col_ind)
rice_bj.columns = [cols[x] for x in col_ind]
att_d = [
"Domestic Consumption",
"Imports",
"Production",
"Rough Production",
"Total Distribution"
"Total Supply",
"TY Imports"]
rice_bj = rice_bj.loc[rice_bj[cols[8]].isin(att_d)]
rice_bj['Total en tonne'] = rice_bj['Total en tonne']*1000 # convert unit to tonne
rice_bj = rice_bj.replace("Imports", "Importation")
rice_bj.head()

Country_Name  Année Attribute_Description  Total en tonne
2         Benin   1960  Domestic Consumption          7000.0
5         Benin   1960           Importation          6000.0
7         Benin   1960            Production          1000.0
8         Benin   1960      Rough Production          2000.0
13        Benin   1960            TY Imports          3000.0

### Estimation des besoins de riz à partir de données historiques (mais incomplètes)

In [151]:
# data from worldbank
import numpy as np
years = ["1959","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017", "2018"]
pop = ["2400127","2431622","2465867","2502896","2542859","2585965","2632356","2682159","2735307","2791590","2850661","2912340","2976572","3043567","3113675","3187412","3265165","3347173","3433439","3523938","3618526","3717165","3820128","3927714","4039949","4156819","4278501","4404506","4535263","4672852","4820016","4978496","5149499","5331803","5521763","5714220","5905558","6094259","6281639","6470265","6664098","6865951","7076733","7295394","7520555","7750004","7982225","8216896","8454791","8696916","8944706","9199259","9460802","9729160","10004451","10286712","10575952","10872298","11002578", "11362269"]
years = [int(x) for x in years]
pop = [int(x) for x in pop]
y_map = dict((k,i) for i,k in enumerate(years))
riz_needed_hist = [[2005, 120000],
[2007, 160000],
[2008, 180000],
[2009, 205000],
[2010, 210000],
[2011, 220000],
[2012, 220000],
[2013, 225000],
[2014, 230000]] # source: http://roppa-afrique.org/IMG/pdf/rapport_etude_don_riz_ben_in.pdf
# trouver la pente étant donnée la population
raid = np.max([x[1]*1.0/pop[y_map[x[0]]] for x in riz_needed])

In [152]:
# predicted total population for 35kg of rice per pers per year
whole_pop_pred = [int(np.ceil(x*raid)/35.0)*1000 for i, x in enumerate(pop)]
whole_pop_pred
# seem reasonable given the actual population 
# and assuming that most toddler / old people do not eat rice


[1594000,
 1615000,
 1638000,
 1662000,
 1689000,
 1718000,
 1748000,
 1782000,
 1817000,
 1854000,
 1893000,
 1934000,
 1977000,
 2022000,
 2068000,
 2117000,
 2169000,
 2223000,
 2281000,
 2341000,
 2404000,
 2469000,
 2538000,
 2609000,
 2684000,
 2761000,
 2842000,
 2926000,
 3013000,
 3104000,
 3202000,
 3307000,
 3421000,
 3542000,
 3668000,
 3796000,
 3923000,
 4049000,
 4173000,
 4298000,
 4427000,
 4561000,
 4701000,
 4847000,
 4996000,
 5149000,
 5303000,
 5459000,
 5617000,
 5778000,
 5942000,
 6111000,
 6285000,
 6464000,
 6646000,
 6834000,
 7026000,
 7223000,
 7310000,
 7549000]

In [153]:
riz_need_pred = [(years[i], 'Besoin minimum', int(np.ceil(x*raid)/1000)*1000) for i, x in enumerate(pop)] #arrondir
df_need_rice = pd.DataFrame(riz_need_pred,columns=['Année', 'Attribute_Description', 'Total en tonne'])
tot_df = pd.concat([rice_bj, df_need_rice])
tot_df = tot_df.loc[tot_df['Attribute_Description'].isin(['Production', 'Importation', 'Besoin minimum'])]
tot_df = tot_df.loc[tot_df['Année']>=1990]

In [154]:
lgd =  {"font": {
        "size": 12
      },"data":False, "size": [20, 50]}

lp = LinePlot(id='Attribute_Description', x={'value':'Année', 'grid': False}, y={'value':'Total en tonne'}, legend=lgd, width=600, container_id="ricebenin")
print lp.dump_html(tot_df)


        <script src='//d3plus.org/js/d3.js' type='text/javascript'></script>
	<script src='//d3plus.org/js/d3plus.js' type='text/javascript'></script>
        <div id='ricebenin'></div>
        <style>
        div#ricebenin{
             width: 600px;
             height: 400px;
        }
        
        </style>
        <script>
            
        (function (){
            
            var viz_data = [{"Ann\u00e9e":1990,"Attribute_Description":"Importation","Country_Name":"Benin","Total en tonne":30000.0},{"Ann\u00e9e":1990,"Attribute_Description":"Production","Country_Name":"Benin","Total en tonne":6000.0},{"Ann\u00e9e":1991,"Attribute_Description":"Importation","Country_Name":"Benin","Total en tonne":30000.0},{"Ann\u00e9e":1991,"Attribute_Description":"Production","Country_Name":"Benin","Total en tonne":6000.0},{"Ann\u00e9e":1992,"Attribute_Description":"Importation","Country_Name":"Benin","Total en tonne":50000.0},{"Ann\u00e9e":1992,"Attribute_Description":"Production","Country